In [24]:
#Data is generated is using the code used before for generating emotion recognition dataset
#Here I'm using FisherFaceRecognizer to train my model for anti-spoofing (liveness of the face)
#Since it is a binary classification unlike emotion recognition it works quite well with limited dataset

import cv2
import glob
import random
import numpy as np
from shutil import copyfile

names=["real","fake"] 
dest_url="C:\\SPB_Data\\RTSP\\Expt3(test)\\dataset"
fishface = cv2.face.FisherFaceRecognizer_create() #Initialize fisher face classifier
data = {}

def get_files(emotion): #Define function to get file list, randomly shuffle it and split 80/20
    files = glob.glob("%s\\%s\\*" %(dest_url,emotion))
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] #get first 80% of file list
    prediction = files[-int(len(files)*0.2):] #get last 20% of file list
    return training, prediction
def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for name in names:
        training, prediction = get_files(name)
        #Append data to training and prediction list, and generate labels 0-7
        for item in training:
            image = cv2.imread(item) #open image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #convert to grayscale
            training_data.append(gray) #append image array to training data list
            training_labels.append(names.index(name))
        for item in prediction: #repeat above process for prediction set
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            prediction_data.append(gray)
            prediction_labels.append(names.index(name))
    return training_data, training_labels, prediction_data, prediction_labels
def run_recognizer():
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
    print("training fisher face classifier")
    print("size of training set is:", len(training_labels), "images")
    fishface.train(training_data, np.asarray(training_labels))
    print("predicting classification set")
    cnt = 0
    correct = 0
    incorrect = 0
    for image in prediction_data:
        pred, conf = fishface.predict(image)
        if pred == prediction_labels[cnt]:
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect))

metascore = []
for i in range(0,1):
    correct = run_recognizer()
    print("got", correct, "percent correct!")
    metascore.append(correct)
print("\n\nend score:", np.mean(metascore), "percent correct!")

training fisher face classifier
size of training set is: 520 images
predicting classification set
got 96.89922480620154 percent correct!


end score: 96.89922480620154 percent correct!


In [25]:
cap = cv2.VideoCapture(0)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('output.avi', fourcc, 30.0, (640,480))

emotions = ["real", "fake"]
while(True):
    ret, frame = cap.read()
    cv2.waitKey(50)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     gray = cv2.equalizeHist(gray)
    
    faceDet = cv2.CascadeClassifier("G:\Github\opencv\data\haarcascades\haarcascade_frontalface_alt.xml")
    face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
    
    for (x,y,w,h) in face:
        cv2.rectangle(frame, (x, y), (x+w,y+h), (0, 255, 0), 2)
        gray1=gray[y:y+h,x:x+w]
        out = cv2.resize(gray1, (100, 100))
        pred, conf = fishface.predict(out)
#         print(emotions[pred])
        cv2.putText(frame, emotions[pred], (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    
    output.write(frame)
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
output.release()
cv2.destroyAllWindows()